In [1]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add the src directory to the path so we can import our modules
sys.path.append('/home/mg873uh/Projects_kb/baseline-codebase/src')

from data.datamodule import PretrainDataModule
from data.dataset import PretrainDatasetCombinedPatient
from data.pretrain_split import get_pretrain_split_config
from augmentations.augmentation_composer import get_pretrain_augmentations, get_val_augmentations
from utils.utils import SimplePathConfig

print("Imports successful!")

Imports successful!


In [2]:
dataset = PretrainDatasetCombinedPatient(["sub_1105", "sub_11064"], (96, 96, 96), "/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/", (96, 96, 96), None)

sub_11064_ses_1_dwi.npy
Processing file: sub_11064_ses_1_dwi.npy for patient sub_11064, session 1, modality dwi
sub_1105_ses_1_dwi.npy
Processing file: sub_1105_ses_1_dwi.npy for patient sub_1105, session 1, modality dwi
sub_11064_ses_1_dwi_2.npy
Processing file: sub_11064_ses_1_dwi_2.npy for patient sub_11064, session 1, modality dwi
sub_1105_ses_1_t2.npy
Processing file: sub_1105_ses_1_t2.npy for patient sub_1105, session 1, modality t2
sub_1105_ses_1_t1.npy
Processing file: sub_1105_ses_1_t1.npy for patient sub_1105, session 1, modality t1
sub_1105_ses_2_t1.npy
Processing file: sub_1105_ses_2_t1.npy for patient sub_1105, session 2, modality t1
sub_11064_ses_1_t1.npy
Processing file: sub_11064_ses_1_t1.npy for patient sub_11064, session 1, modality t1
sub_1105_ses_2_t2.npy
Processing file: sub_1105_ses_2_t2.npy for patient sub_1105, session 2, modality t2
sub_11064_ses_1_flair.npy
Processing file: sub_11064_ses_1_flair.npy for patient sub_11064, session 1, modality flair
sub_1105_ses

In [3]:
dataset.all_files

{'sub_11064/ses_1': {<ModalityIndex.DWI: 4>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/sub_11064_ses_1_dwi_2.npy',
  <ModalityIndex.T1: 0>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/sub_11064_ses_1_t1.npy',
  <ModalityIndex.FLAIR: 2>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/sub_11064_ses_1_flair.npy'},
 'sub_1105/ses_1': {<ModalityIndex.DWI: 4>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/sub_1105_ses_1_dwi.npy',
  <ModalityIndex.T2: 1>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/sub_1105_ses_1_t2.npy',
  <ModalityIndex.T1: 0>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/sub_1105_ses_1_t1.npy'},
 'sub_1105/ses_2': {<ModalityIndex.T1: 0>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/sub_1105_ses_2_t1.npy',
  <ModalityIndex.T2: 1>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/sub_1105_ses_2_t2.npy',
  <ModalityIndex.DWI: 4>: '/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO6